In [57]:
import pandas as pd
import numpy as np
import torch

from matplotlib import pyplot as plt 

import time

In [58]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchvision import datasets, models, transforms, utils
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T
import torchvision

import os
import copy

from skimage import io, transform

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   

In [59]:
from pydub import AudioSegment
from pydub.playback import play
import librosa

In [60]:
print(torchvision.__version__)

0.15.2+cu118


In [61]:
torch.cuda.is_available()

True

In [62]:
for model_dir in os.listdir("./"):
    if model_dir.split(".")[-1] != "pt":
        continue
    print(model_dir)   
        



best.pt
CNN_0.962963.pt
MLP_0.761364.pt
MLP_0.851064.pt
MLP_0.872340.pt
MLP_0.888889new_model3.pt
MLP_0.893617.pt
MLP_0.914894.pt
MLP_0.919753.pt
MLP_0.919753new_model2.pt
MLP_0.932099new_model3.pt
MLP_0.936170.pt
MLP_0.936170_best.pt
MLP_0.936170_old.pt
MLP_0.938272new_model3.pt


In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## first model of ensemble model

In [64]:
class MLP_Audio_Model2(torch.nn.Module):

    def __init__(self):
        super(MLP_Audio_Model2, self).__init__()


        self.linear1 = torch.nn.Linear(2790, 256)
        self.bachNoem1 = nn.BatchNorm1d(256)
        self.activation1 = torch.nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)

        self.linear2 = torch.nn.Linear(256, 64)
        self.bachNoem2 = nn.BatchNorm1d(64)
        self.activation2 = torch.nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.5)

#         self.linear3 = torch.nn.Linear(128, 32)
#         self.bachNoem3 = torch.nn.BatchNorm1d(32)
#         self.dropout3 = nn.Dropout(p=0.1)
#         self.activation3 = torch.nn.ReLU()

        # self.linear4 = torch.nn.Linear(128, 32)
        # self.bachNoem4 = torch.nn.BatchNorm1d(32)
        # self.dropout4 = nn.Dropout(p=0.3)
        # self.activation4 = torch.nn.ReLU()
        
#         self.linear6 = torch.nn.Linear(64, 32)
#         self.bachNoem6 = torch.nn.BatchNorm1d(32)
#         self.dropout6 = nn.Dropout(p=0.1)
#         self.activation6 = torch.nn.ReLU()
        
#         self.linear7 = torch.nn.Linear(32, 16)
#         self.bachNoem7 = torch.nn.BatchNorm1d(16)
#         self.dropout7 = nn.Dropout(p=0.1)
#         self.activation7 = torch.nn.ReLU()
        
#         self.linear8 = torch.nn.Linear(32, 16)
#         self.bachNoem8 = torch.nn.BatchNorm1d(16)
#         self.dropout8 = nn.Dropout(p=0.2)
#         self.activation8 = torch.nn.ReLU()
        
#         self.linear9 = torch.nn.Linear(32, 16)
#         self.bachNoem9 = torch.nn.BatchNorm1d(16)
#         self.dropout9 = nn.Dropout(p=0.2)
#         self.activation9 = torch.nn.ReLU()

        self.linear5 = torch.nn.Linear(64, 4)

        self.softmax = torch.nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.bachNoem1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        x = self.linear2(x)
        x = self.bachNoem2(x)
        x = self.activation2(x)
        x = self.dropout2(x)

#         x = self.linear3(x)
#         x = self.bachNoem3(x)
#         x = self.dropout3(x)
#         x = self.activation3(x)

        # x = self.linear4(x)
        # x = self.bachNoem4(x)
        # x = self.dropout4(x)
        # x = self.activation4(x)        

#         x = self.linear6(x)
#         x = self.bachNoem6(x)
#         # x = self.dropout6(x)
#         x = self.activation6(x)      
        
#         x = self.linear7(x)
#         x = self.bachNoem7(x)
#         x = self.dropout7(x)
#         x = self.activation7(x)
        
        x = self.linear5(x)

        x = self.softmax(x)
        return x

MLP_model1= MLP_Audio_Model2()

print('The model:')
print(MLP_model1)

# print('\n\nModel params:')
# for param in MLP_model.parameters():
#     print(param)

The model:
MLP_Audio_Model2(
  (linear1): Linear(in_features=2790, out_features=256, bias=True)
  (bachNoem1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=256, out_features=64, bias=True)
  (bachNoem2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (linear5): Linear(in_features=64, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


In [65]:
current_dir = os.getcwd()
model_path = current_dir + '\\' + "MLP_0.919753new_model2.pt"

test_model_1 = MLP_Audio_Model2()

test_model_1 = torch.load(model_path)


# test_model = torch.load(model_path)

test_model_1.eval()
test_model_1 = test_model_1.to(device)

## second model of ensemble model

In [66]:
class MLP_Audio_Model3(torch.nn.Module):

    def __init__(self):
        super(MLP_Audio_Model3, self).__init__()


        self.linear1 = torch.nn.Linear(2790, 512)
        self.bachNoem1 = nn.BatchNorm1d(512)
        self.activation1 = torch.nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)

        # self.linear2 = torch.nn.Linear(1024, 512)
        # self.bachNoem2 = nn.BatchNorm1d(512)
        # self.activation2 = torch.nn.ReLU()
        # self.dropout2 = nn.Dropout(p=0.5)

        self.linear3 = torch.nn.Linear(512, 128)
        self.bachNoem3 = torch.nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(p=0.5)
        self.activation3 = torch.nn.ReLU()

#         self.linear4 = torch.nn.Linear(256, 128)
#         self.bachNoem4 = torch.nn.BatchNorm1d(128)
#         self.dropout4 = nn.Dropout(p=0.3)
#         self.activation4 = torch.nn.ReLU()
        
        self.linear6 = torch.nn.Linear(128, 64)
        self.bachNoem6 = torch.nn.BatchNorm1d(64)
        self.dropout6 = nn.Dropout(p=0.5)
        self.activation6 = torch.nn.ReLU()
        
        # self.linear7 = torch.nn.Linear(64, 32)
        # self.bachNoem7 = torch.nn.BatchNorm1d(32)
        # self.dropout7 = nn.Dropout(p=0.5)
        # self.activation7 = torch.nn.ReLU()
        
        self.linear8 = torch.nn.Linear(64, 16)
        self.bachNoem8 = torch.nn.BatchNorm1d(16)
        self.dropout8 = nn.Dropout(p=0.5)
        self.activation8 = torch.nn.ReLU()
        
#         self.linear9 = torch.nn.Linear(32, 16)
#         self.bachNoem9 = torch.nn.BatchNorm1d(16)
#         self.dropout9 = nn.Dropout(p=0.2)
#         self.activation9 = torch.nn.ReLU()

        self.linear5 = torch.nn.Linear(16, 4)

        self.softmax = torch.nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.bachNoem1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        # x = self.linear2(x)
        # x = self.bachNoem2(x)
        # x = self.activation2(x)
        # x = self.dropout2(x)

        x = self.linear3(x)
        x = self.bachNoem3(x)
        x = self.activation3(x)
        x = self.dropout3(x)

        # x = self.linear4(x)
        # x = self.bachNoem4(x)
        # x = self.activation4(x)
        # x = self.dropout4(x)

        x = self.linear6(x)
        x = self.bachNoem6(x)
        x = self.activation6(x)
        x = self.dropout6(x)
        
        # x = self.linear7(x)
        # x = self.bachNoem7(x)
        # x = self.activation7(x)
        # x = self.dropout7(x)
        
        x = self.linear8(x)
        x = self.bachNoem8(x)
        x = self.activation8(x)
        x = self.dropout8(x)   
        
        
        x = self.linear5(x)

        x = self.softmax(x)
        return x

MLP_model = MLP_Audio_Model3()

print('The model:')
print(MLP_model)

# print('\n\nModel params:')
# for param in MLP_model.parameters():
#     print(param)

The model:
MLP_Audio_Model3(
  (linear1): Linear(in_features=2790, out_features=512, bias=True)
  (bachNoem1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear3): Linear(in_features=512, out_features=128, bias=True)
  (bachNoem3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (activation3): ReLU()
  (linear6): Linear(in_features=128, out_features=64, bias=True)
  (bachNoem6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout6): Dropout(p=0.5, inplace=False)
  (activation6): ReLU()
  (linear8): Linear(in_features=64, out_features=16, bias=True)
  (bachNoem8): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout8): Dropout(p=0.5, inplace=False)
  (activation8): ReLU()
  (linear5): Linear(in_features=16, out_features=4, bia

In [67]:
current_dir = os.getcwd()
model_path = current_dir + '\\' + "MLP_0.888889new_model3.pt"

test_model_2 = MLP_Audio_Model3()

test_model_2 = torch.load(model_path)


test_model_2.eval()
test_model_2 = test_model_2.to(device)
print(test_model_2)

MLP_Audio_Model3(
  (linear1): Linear(in_features=2790, out_features=512, bias=True)
  (bachNoem1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear3): Linear(in_features=512, out_features=128, bias=True)
  (bachNoem3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (activation3): ReLU()
  (linear6): Linear(in_features=128, out_features=64, bias=True)
  (bachNoem6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout6): Dropout(p=0.5, inplace=False)
  (activation6): ReLU()
  (linear8): Linear(in_features=64, out_features=16, bias=True)
  (bachNoem8): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout8): Dropout(p=0.5, inplace=False)
  (activation8): ReLU()
  (linear5): Linear(in_features=16, out_features=4, bias=True)
  (

## third model of ensemble model

In [68]:
class MLP_Audio_Model(torch.nn.Module):

    def __init__(self):
        super(MLP_Audio_Model, self).__init__()


        self.linear1 = torch.nn.Linear(2790, 512)
        self.bachNoem1 = nn.BatchNorm1d(512)
        self.activation1 = torch.nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)

        self.linear2 = torch.nn.Linear(512, 128)
        self.bachNoem2 = nn.BatchNorm1d(128)
        self.activation2 = torch.nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.5)

#         self.linear3 = torch.nn.Linear(128, 32)
#         self.bachNoem3 = torch.nn.BatchNorm1d(32)
#         self.dropout3 = nn.Dropout(p=0.1)
#         self.activation3 = torch.nn.ReLU()

        self.linear4 = torch.nn.Linear(128, 32)
        self.bachNoem4 = torch.nn.BatchNorm1d(32)
        self.dropout4 = nn.Dropout(p=0.3)
        self.activation4 = torch.nn.ReLU()
        
#         self.linear6 = torch.nn.Linear(64, 32)
#         self.bachNoem6 = torch.nn.BatchNorm1d(32)
#         self.dropout6 = nn.Dropout(p=0.1)
#         self.activation6 = torch.nn.ReLU()
        
#         self.linear7 = torch.nn.Linear(32, 16)
#         self.bachNoem7 = torch.nn.BatchNorm1d(16)
#         self.dropout7 = nn.Dropout(p=0.1)
#         self.activation7 = torch.nn.ReLU()
        
#         self.linear8 = torch.nn.Linear(32, 16)
#         self.bachNoem8 = torch.nn.BatchNorm1d(16)
#         self.dropout8 = nn.Dropout(p=0.2)
#         self.activation8 = torch.nn.ReLU()
        
#         self.linear9 = torch.nn.Linear(32, 16)
#         self.bachNoem9 = torch.nn.BatchNorm1d(16)
#         self.dropout9 = nn.Dropout(p=0.2)
#         self.activation9 = torch.nn.ReLU()

        self.linear5 = torch.nn.Linear(32, 4)

        self.softmax = torch.nn.Softmax(dim = 1)

    def forward(self, x):
        x = self.linear1(x)
        x = self.bachNoem1(x)
        x = self.activation1(x)
        x = self.dropout1(x)

        x = self.linear2(x)
        x = self.bachNoem2(x)
        x = self.activation2(x)
        x = self.dropout2(x)

#         x = self.linear3(x)
#         x = self.bachNoem3(x)
#         x = self.dropout3(x)
#         x = self.activation3(x)

        x = self.linear4(x)
        x = self.bachNoem4(x)
        x = self.dropout4(x)
        x = self.activation4(x)        

#         x = self.linear6(x)
#         x = self.bachNoem6(x)
#         # x = self.dropout6(x)
#         x = self.activation6(x)      
        
#         x = self.linear7(x)
#         x = self.bachNoem7(x)
#         x = self.dropout7(x)
#         x = self.activation7(x)
        
        x = self.linear5(x)

        x = self.softmax(x)
        return x

MLP_model = MLP_Audio_Model()

print('The model:')
print(MLP_model)

# print('\n\nModel params:')
# for param in MLP_model.parameters():
#     print(param)

The model:
MLP_Audio_Model(
  (linear1): Linear(in_features=2790, out_features=512, bias=True)
  (bachNoem1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
  (bachNoem2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (linear4): Linear(in_features=128, out_features=32, bias=True)
  (bachNoem4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (activation4): ReLU()
  (linear5): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


In [69]:
current_dir = os.getcwd()
model_path = current_dir + '\\' + "best.pt"

test_model_3 = MLP_Audio_Model()

test_model_3 = torch.load(model_path)


test_model_3.eval()
test_model_3 = test_model_3.to(device)
print(test_model_3)

MLP_Audio_Model(
  (linear1): Linear(in_features=2790, out_features=512, bias=True)
  (bachNoem1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (linear2): Linear(in_features=512, out_features=128, bias=True)
  (bachNoem2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (linear4): Linear(in_features=128, out_features=32, bias=True)
  (bachNoem4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (activation4): ReLU()
  (linear5): Linear(in_features=32, out_features=4, bias=True)
  (softmax): Softmax(dim=1)
)


In [70]:
import pyaudio
import math
from scipy import signal
import soundfile as sf
import gradio as gr
import sounddevice as sd

## proccess function for inputs(ensemble learning)

In [74]:
SAMPLE_RATE = 16000

def proccess(frame):
    
        dir_dict = {0: "Down", 1: "Left", 2: "Right", 3: "Up"}
        
        tensor = frame

        print(tensor.size())
        
#         target_size = 48000
#         tensor = torch.nn.functional.pad(tensor, (0, target_size - tensor.size(1), 0, 0), 'constant', 0)
        
        print(tensor.size())


        live_transform = T.MFCC(
            sample_rate=SAMPLE_RATE,
            n_mfcc=31,
            melkwargs={"n_fft": 2048  , "hop_length": 512 , "n_mels": 128, "center": False}
        )

        tensor = live_transform(tensor)
        
        tensor = nn.functional.normalize(tensor, dim=-1)

        print(tensor.size())

        tensor = torch.flatten(tensor)

        tensor = torch.reshape(tensor, (1, 2790))
        
        tensor = tensor.to(device)

        output_1 = test_model_1(tensor)
        selected_1 = torch.argmax(output_1.to("cpu"), dim=1)
        output_2 = test_model_2(tensor)
        selected_2 = torch.argmax(output_2.to("cpu"), dim=1)
        output_3 = test_model_3(tensor)
        selected_3 = torch.argmax(output_3.to("cpu"), dim=1)
        
        output = (output_1*3 + output_2*6 + output_3)/10
        
        preds = output
        
        print(output_1 , "\n", output_2, "\n", output_3, "\n",  preds)

        predicted_value = torch.argmax(preds.to("cpu"), dim=1)
        
        if(selected_1[0] == 3 or selected_2 == 3):
            predicted_value = torch.tensor([3], dtype=torch.float16)
            
        if output_1[0][selected_1[0]] > 0.9:
            predicted_value = selected_1
        elif output_2[0][selected_2[0]] > 0.9:
            predicted_value = selected_2
        elif output_2[0][selected_3[0]] > 0.9:
            predicted_value = selected_3
        

        # Return the output as a numpy array
        return dir_dict[predicted_value.detach().numpy().item()]

## real-time block

In [76]:
input_format = pyaudio.paInt16
input_channels = 1
input_sample_rate = 16000  # Changed to 16000 samples per second
chunk_size = 2048
output_format = 'WAV'
output_channels = 1
output_sample_rate = 16000
output_duration = 3  # seconds
num_output_frames = output_sample_rate * output_duration

# Create a PyAudio object
audio = pyaudio.PyAudio()

while True:
    time.sleep(3)
    print('Recording...')

    # Open the microphone stream
    stream = audio.open(format=input_format, channels=input_channels, rate=input_sample_rate, input=True, frames_per_buffer=chunk_size)

    # Start the stream
    since = time.time()
    stream.start_stream()

    # Record 3 seconds of audio
    recorded_frames = []
    for i in range(int(input_sample_rate / chunk_size * output_duration) + 1):
        data = stream.read(chunk_size)
        recorded_frames.append(data)

    # Stop the stream
    time_elapsed = time.time() - since
    print(f"duration: {time_elapsed}")
    stream.stop_stream()
    stream.close()

    print('Processing...')

    # Convert the recorded audio to a NumPy array
    recorded_data = np.frombuffer(b''.join(recorded_frames), dtype=np.int16)
    recorded_data = recorded_data[:48000]
    sd.play(recorded_data, 16000)
    recorded_data = recorded_data.astype(np.float16)
    print(recorded_data.shape)
    
    # Reshape the audio to (1, 48000) size
    audio_tensor = torch.from_numpy(recorded_data).reshape(1, -1)
    print(audio_tensor.size())
    # Process the audio with the pre-trained model
    with torch.no_grad():
        output = proccess(audio_tensor)

    # Print the predicted label
    print(f'Predicted label: {output}')

Recording...
duration: 3.0705764293670654
Processing...
(48000,)
torch.Size([1, 48000])
torch.Size([1, 48000])
torch.Size([1, 48000])
torch.Size([1, 31, 90])
tensor([[0.7939, 0.1124, 0.0773, 0.0165]], device='cuda:0') 
 tensor([[0.6528, 0.0818, 0.1728, 0.0926]], device='cuda:0') 
 tensor([[0.4693, 0.0102, 0.5128, 0.0077]], device='cuda:0') 
 tensor([[0.6768, 0.0838, 0.1781, 0.0613]], device='cuda:0')
Predicted label: [0]
Recording...
duration: 3.0708789825439453
Processing...
(48000,)
torch.Size([1, 48000])
torch.Size([1, 48000])
torch.Size([1, 48000])
torch.Size([1, 31, 90])
tensor([[0.0395, 0.8962, 0.0612, 0.0031]], device='cuda:0') 
 tensor([[0.0503, 0.5540, 0.3752, 0.0205]], device='cuda:0') 
 tensor([[0.0193, 0.0065, 0.9719, 0.0024]], device='cuda:0') 
 tensor([[0.0440, 0.6019, 0.3407, 0.0135]], device='cuda:0')
Predicted label: [1]
Recording...
duration: 3.070493459701538
Processing...
(48000,)
torch.Size([1, 48000])
torch.Size([1, 48000])
torch.Size([1, 48000])
torch.Size([1, 31

KeyboardInterrupt: 